In [34]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.framework import ops
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dropout, Flatten, Dense,Conv2DTranspose,UpSampling2D
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
tf.config.run_functions_eagerly(True)
#from tensorflow.keras.models import load_model
import numpy as np
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
import os
import re

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dropout, Flatten, Dense,Conv2DTranspose,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

In [36]:
# training without normalizing the image (scaling)
dataset_dir = Path('C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset')
image_count = len(list(dataset_dir.glob('*/*.png')))
print(image_count)
#dataset_dir = 'C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset'

checkpoint_path = 'C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras'
best_checkpoint_path = 'C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras'
saved_model_path = 'C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras'

batch_size = 32
img_height = 100
img_width = 89
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
#test_ratio = 0.2
#validation_ratio = 0.2
#batch_size = 50
#epochs = 10
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



14320
Found 14320 files belonging to 12 classes.
Using 11456 files for training.
Found 14320 files belonging to 12 classes.
Using 2864 files for validation.


In [37]:
class_names = train_ds.class_names
print(class_names)

['Direction_left', 'Direction_right', 'Fist', 'Five-palm', 'Four', 'OK', 'One', 'Stop', 'Three', 'Thumbs_down', 'Thumbs_up', 'Two']


In [38]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 100, 89, 3)
(32,)


In [39]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [40]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)
num_classes = len(class_names)
print(num_classes)

12


In [41]:
input_layer = Input(shape=[ 100, 89, 3], name='input')
convnet = Conv2D(32,2, activation='relu')(input_layer)
convnet =MaxPooling2D( 2,2)(convnet)
convnet =Conv2D( 64,( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)
convnet =Conv2D( 128, ( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)

convnet =Conv2D( 256, ( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)

convnet = Flatten()(convnet) 
convnet = Dense(128,activation='relu')(convnet)
convnet=Dropout(0.50)(convnet)
convnet=Dense(12,activation='softmax')(convnet)
model = Model(inputs=input_layer, outputs=convnet)
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='SparseCategoricalCrossentropy', metrics=['accuracy'])





#model = Model(inputs=input_layer, outputs=output_layer)

model.save('C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras')

In [42]:
from keras.callbacks import ModelCheckpoint

snapshot_step = 100
checkpoint = ModelCheckpoint(filepath='C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModel_{epoch:02d}.keras', save_best_only=True, period=snapshot_step)
model.fit(train_ds, epochs=10,
           validation_data=val_ds,
        callbacks=[checkpoint])

model.save('C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras')

Epoch 1/10
358/358 [==============================] - 249s 673ms/step - loss: 1.2201 - accuracy: 0.8974 - val_loss: 0.0125 - val_accuracy: 0.9962
Epoch 2/10
358/358 [==============================] - 131s 367ms/step - loss: 0.0619 - accuracy: 0.9849 - val_loss: 0.0557 - val_accuracy: 0.9850
Epoch 3/10
358/358 [==============================] - 94s 264ms/step - loss: 0.0392 - accuracy: 0.9907 - val_loss: 0.0016 - val_accuracy: 0.9993
Epoch 4/10
358/358 [==============================] - 121s 337ms/step - loss: 0.0275 - accuracy: 0.9935 - val_loss: 0.0024 - val_accuracy: 0.9993
Epoch 5/10
358/358 [==============================] - 218s 610ms/step - loss: 0.0294 - accuracy: 0.9948 - val_loss: 0.0056 - val_accuracy: 0.9990
Epoch 6/10
358/358 [==============================] - 217s 605ms/step - loss: 0.0190 - accuracy: 0.9966 - val_loss: 5.9039e-07 - val_accuracy: 1.0000
Epoch 7/10
358/358 [==============================] - 215s 600ms/step - loss: 0.0169 - accuracy: 0.9968 - val_loss: 8.278

In [43]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 100, 89, 3)]      0         
                                                                 
 conv2d_20 (Conv2D)          (None, 99, 88, 32)        416       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 49, 44, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 48, 43, 64)        8256      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 24, 21, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 23, 20, 128)       3289

In [30]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


ValueError: Shapes (32, 1) and (32, 12) are incompatible

In [44]:
sunflower_url =  Path('d:/movies/30_l=3.png')


img = tf.keras.utils.load_img(
    sunflower_url, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 76ms/step
This image most likely belongs to Five-palm with a 19.82 percent confidence.


In [5]:
for d in alldirs:
    for img in os.listdir(d):
        label = re.findall('[l]=[0-9]+', img)
        label = label[0][2:]
        
        current_img = f"{d}/{img}"
        image = cv2.imread(current_img)
        resized_image = cv2.resize(image, (100, 89))
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        
        all_images.append(gray_image.reshape(89, 100, 1))
        class_ids.append(label)
    print(f"{d} done.")
            
all_images = np.array(all_images)
class_ids = np.array(class_ids)

#print(len(all_images))
#print(len(class_ids))


C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Direction_left done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Direction_right done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Fist done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Five-palm done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Four done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/OK done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/One done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Stop done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Three done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Thumbs_down done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Thumbs_up done.
C:/deva/Miniproject/automation-using-hand-gestures-master/Dataset/Two done.


In [6]:
x_train, x_test, y_train, y_test = train_test_split(all_images, class_ids, test_size=test_ratio)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_ratio)

x_train.shape





(1472, 89, 100, 1)

In [7]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
y_validation = to_categorical(y_validation, n_classes)


In [18]:
ops.reset_default_graph()
#                           [batch, height, width, in_channels]
#input_layer = Input(shape=(89, 100, 1), name='input')
input_layer = Input(shape=[ 89, 100, 1], name='input')
convnet = Conv2D(32,2, activation='relu')(input_layer)
convnet =MaxPooling2D( 2,2)(convnet)
convnet =Conv2D( 64,( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)
convnet =Conv2D( 128, ( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)

convnet =Conv2D( 256, ( 2,2), activation='relu')(convnet)
convnet = MaxPooling2D( 2,2)(convnet)
convnet = Conv2DTranspose( 256, (2,2), activation='relu')(convnet)
convnet = UpSampling2D(  (2,2))(convnet)

convnet = Conv2DTranspose( 128, (2,2), activation='relu')(convnet)
convnet = UpSampling2D( (2,2))(convnet)

convnet = Conv2DTranspose( 64,  (2,2), activation='relu')(convnet)
convnet = UpSampling2D( (2,2))(convnet)
convnet = Flatten()(convnet) 
convnet = Dense(1000,activation='relu')(convnet)
convnet=Dropout(0.75)(convnet)
convnet=Dense(12,activation='softmax')(convnet)
#model = Sequential() 
#model.add(Conv2D(32, (3,3),padding='same' ,activation='relu',input_shape=input_shape))
#model.add(MaxPooling2D((2,2),padding='same'))
#model.add(Conv2D(64, (3,3)   ,padding='same' ,activation='relu'))
#model.add(MaxPooling2D((2,2),padding='same'))
#model.add(Conv2D(128, (3,3)   ,padding='same' ,activation='relu'))
#model.add(MaxPooling2D((2,2),padding='same'))
###model.add(Conv2D(256, (3,3)   ,padding='same' ,activation='relu'))
#model.add(MaxPooling2D((2,2),padding='same'))
#model.add(Flatten())
#model.add(Dense(1000, activation='relu'))
#model.add(Dropout(0.75))
#   model.add( Dense(12, activation='softmax'))
model = Model(inputs=input_layer, outputs=convnet)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])





#model = Model(inputs=input_layer, outputs=output_layer)

model.save('C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras')


In [19]:
from keras.callbacks import ModelCheckpoint
x_train, y_train = shuffle(x_train, y_train, random_state=0)
snapshot_step = 100
checkpoint = ModelCheckpoint(filepath='C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModel_{epoch:02d}.keras', save_best_only=True, period=snapshot_step)
model.fit(x_train, y_train, epochs=10,
           validation_data=(x_validation, y_validation),
        callbacks=[checkpoint])

model.save('C:/deva/Miniproject/automation-using-hand-gestures-master/TrainedModel/Gesture12RecognitionModelBest.keras')

Epoch 1/10
46/46 [==============================] - 121s 3s/step - loss: 7.9219 - accuracy: 0.5577 - val_loss: 0.3385 - val_accuracy: 0.9130
Epoch 2/10
46/46 [==============================] - 119s 3s/step - loss: 0.2275 - accuracy: 0.9348 - val_loss: 0.0478 - val_accuracy: 0.9918
Epoch 3/10
46/46 [==============================] - 118s 3s/step - loss: 0.0432 - accuracy: 0.9851 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 4/10
46/46 [==============================] - 116s 3s/step - loss: 0.0265 - accuracy: 0.9925 - val_loss: 0.0186 - val_accuracy: 0.9946
Epoch 5/10
46/46 [==============================] - 115s 2s/step - loss: 0.0569 - accuracy: 0.9837 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 6/10
46/46 [==============================] - 113s 2s/step - loss: 0.0505 - accuracy: 0.9830 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 7/10
46/46 [==============================] - 113s 2s/step - loss: 0.0816 - accuracy: 0.9776 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 8/10
46

In [21]:
print(model.summary())

score = model.evaluate(x_test, y_test)
print(score)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 89, 100, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 88, 99, 32)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 44, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 43, 48, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 21, 24, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 23, 128)       32896 

In [33]:
sunflower_url =  Path('D:/movies/1_l=1.png')


img = tf.keras.utils.load_img(
    sunflower_url, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 55ms/step
This image most likely belongs to Thumbs_down with a 19.82 percent confidence.
